# Pré-processamento do Dataset FakeBR

Este notebook realiza o pré-processamento do dataset FakeBR, incluindo limpeza de dados, normalização de texto e preparação para análise de fairness.

In [35]:
# Importações necessárias
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn


# Verificação das versões dos pacotes principais
print("Versões dos pacotes:")
print(f"   - pandas: {pd.__version__}")
print(f"   - numpy: {np.__version__}")
print(f"   - nltk: {nltk.__version__}")
print(f"   - sklearn: {sklearn.__version__}")
print(f"   - matplotlib: {matplotlib.__version__}")
print(f"   - seaborn: {sns.__version__}")

# Download recursos do NLTK se necessário
print("\nVerificando recursos do NLTK...")
try:
    nltk.data.find('tokenizers/punkt')
    print("Tokenizer punkt já disponível")
except LookupError:
    print("Baixando tokenizer punkt...")
    nltk.download('punkt')
    
try:
    nltk.data.find('corpora/stopwords')
    print("Stopwords já disponíveis")
except LookupError:
    print("Baixando stopwords...")
    nltk.download('stopwords')

print("Todas as importações concluídas com sucesso!")

Versões dos pacotes:
   - pandas: 2.3.3
   - numpy: 2.3.4
   - nltk: 3.9.2
   - sklearn: 1.7.2
   - matplotlib: 3.10.7
   - seaborn: 0.13.2

Verificando recursos do NLTK...
Tokenizer punkt já disponível
Stopwords já disponíveis
Todas as importações concluídas com sucesso!


## 1. Carregamento de Dataset e Análise Exploratória: 
Primeiro vamos carregar o dataset e depois realizar uma análise exploratória dos dados.

In [20]:
import os
import pandas as pd
from IPython.display import display

# Caminhos dos arquivos
xlsx_path = "../data/raw/fakeBr.xlsx"
csv_path = "../data/processed/fakeBr_dataset.csv"

# 1Verifica se existe o Excel e converte para CSV (só se ainda não existir)
if os.path.exists(xlsx_path) and not os.path.exists(csv_path):
    print("Convertendo Excel (.xlsx) para CSV...")
    df_temp = pd.read_excel(xlsx_path)
    df_temp.to_csv(csv_path, index=False, encoding="utf-8")
    print("Conversão concluída!")

# Carrega o CSV se existir
if os.path.exists(csv_path):
    print("Dataset encontrado!")
    df = pd.read_csv(csv_path)

    # Informações básicas
    print(f"Shape do dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")
    print(f"Colunas disponíveis: {df.columns.tolist()}")
    
    # Visualizar amostra
    print("\n" + "="*50)
    print("Primeiras 3 linhas:")
    display(df.head(3))

# Caso o dataset não exista
else:
    print("Dataset não encontrado!")

    # Cria dados de exemplo (para testes)
    print("\n Criando dados de exemplo para teste...")
    df = pd.DataFrame({
        'text': [
            'Esta é uma notícia verdadeira sobre política',
            'FAKE: Notícia falsa inventada para teste',
            'Notícia real sobre economia brasileira'
        ],
        'label': ['real', 'fake', 'real']
    })
    print("Dados de exemplo criados!")
    display(df)


Dataset encontrado!
Shape do dataset: 7200 linhas x 27 colunas
Colunas disponíveis: ['conteudo', 'autor', 'link', 'categoria', 'data_publicacao', 'n_tokens', 'n_palavras_sem_pontuacao', 'n_tipos', 'n_links_na_noticia', 'n_palavras_maiusculas', 'n_verbos', 'n_verbos_subjuntivo_imperativo', 'n_substantivos', 'n_adjetivos', 'n_adverbios', 'n_verbos_modais', 'n_pronomes_pessoas_singulares_primeiro_segundo', 'n_pronomes_plurais_primeiro', 'n_pronomes', 'pausas', 'n_caracteres', 'compr_medio_frase', 'compr_medio_palavra', 'porcent_noticias_erros_ortograficos', 'emotividade', 'diversidade', 'classificacao']

Primeiras 3 linhas:
Shape do dataset: 7200 linhas x 27 colunas
Colunas disponíveis: ['conteudo', 'autor', 'link', 'categoria', 'data_publicacao', 'n_tokens', 'n_palavras_sem_pontuacao', 'n_tipos', 'n_links_na_noticia', 'n_palavras_maiusculas', 'n_verbos', 'n_verbos_subjuntivo_imperativo', 'n_substantivos', 'n_adjetivos', 'n_adverbios', 'n_verbos_modais', 'n_pronomes_pessoas_singulares_pri

,conteudo,autor,link,categoria,data_publicacao,n_tokens,n_palavras_sem_pontuacao,n_tipos,n_links_na_noticia,n_palavras_maiusculas,...,n_pronomes_plurais_primeiro,n_pronomes,pausas,n_caracteres,compr_medio_frase,compr_medio_palavra,porcent_noticias_erros_ortograficos,emotividade,diversidade,classificacao
0,Kátia Abreu diz que vai colocar sua expulsão e...,mrk,https://ceticismopolitico.com/2017/11/30/katia...,politica,2017-11-30,211,185,120,0.0,6,...,0,26,2.0000,815,14.2308,4.40541,0.000000,0.263158,0.648649,1
1,"Dr. Ray peita Bolsonaro, chama-o de conservad...",NaN,https://ceticismopolitico.com/2017/11/24/dr-ra...,politica,2017-11-24,289,254,163,0.0,0,...,0,20,2.5000,1205,18.1429,4.74409,0.007874,0.241667,0.641732,1
2,Reinaldo Azevedo desmascarado pela Polícia Fed...,NaN,https://afolhabrasil.com.br/politica/reinaldo-...,politica,2017-05-23,304,275,170,0.0,0,...,0,18,1.8125,1344,17.1875,4.88727,0.003636,0.127820,0.618182,1


Esse bloco é o início da EDA (Exploratory Data Analysis) do projeto de detecção de fake news. Ele permite validar se o dataset Fake.Br foi carregado corretamente, se há dados faltantes, se as classes estão balanceadas e se os tipos de dados estão adequados antes da análise de vieses.

| Dataset         | Nome da coluna  | Significado no arquivo original         |
| --------------- | --------------- | --------------------------------------- |
| **Fake.Br**     | `classificacao` | `1 = fake news` e `0 = real` (inverso!) |

No Fake.Br, o código precisaria inverter o mapeamento, vamos deixar tudo padronizado com segurança

In [21]:
print("Análise exploratória dos dados iniciada...")
print("="*50)

#Infos gerais do dataset
print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]}")
print(f"Memória: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")

#Verificando valores ausentes
print(f"Valores ausentes por coluna:")
print(df.isnull().sum())

if 'label' in df.columns:
    print(f"Distribuição de labels:")
    print(df['label'].value_counts())
    print(f"\n Proporção")
    print(df['label'].value_counts(normalize=True).round(3))

print(f"Mostra o tipo de dados")
print(df.dtypes)

Análise exploratória dos dados iniciada...
Dimensões: 7200 linhas x 27
Memória: 48.13 MB
Valores ausentes por coluna:
conteudo                                             0
autor                                             3601
link                                                 0
categoria                                            0
data_publicacao                                      0
n_tokens                                             0
n_palavras_sem_pontuacao                             0
n_tipos                                              0
n_links_na_noticia                                1393
n_palavras_maiusculas                                0
n_verbos                                             0
n_verbos_subjuntivo_imperativo                       0
n_substantivos                                       0
n_adjetivos                                          0
n_adverbios                                          0
n_verbos_modais                                      0
n_

In [ ]:
# Análise detalhada da coluna target (classificacao)
print("Análise da Variável Target:")
print("="*50)

# Contagem e proporção
print(f"Distribuição das classes:")
if 'classificacao' in df.columns:
    classificacao_counts = df['classificacao'].value_counts()
elif 'Classe' in df.columns:
    classificacao_counts = df['Classe'].value_counts()
else:
    raise KeyError("Nenhuma coluna de classificação encontrada (esperado 'classificacao' ou 'Classe').")

print(classificacao_counts)

print(f"\nProporção das classes:")
classificacao_prop = classificacao_counts / classificacao_counts.sum()
print(classificacao_prop.round(3))

# Mapeamento automático conforme o dataset
if 'classificacao' in df.columns:
    print("Dataset Fake.Br detectado — invertendo para o padrão (1=real, 0=fake).")
    df['REAL'] = df['classificacao'].map({1: 0, 0: 1})
    df['label_texto'] = df['REAL'].map({1: 'real', 0: 'fake'})
else:
    raise KeyError("Coluna de classificação não encontrada.")

# Exibir distribuição com os labels textuais
print(f"\nDistribuição com labels de texto:")
print(df['label_texto'].value_counts())

# Verificar balanceamento
total = len(df)
for classe, count in classificacao_counts.items():
    pct = (count/total) * 100
    status = "Balanceado" if 40 <= pct <= 60 else "Desbalanceado"
    print(f"\nClasse {classe}: {count} amostras ({pct:.1f}%) - {status}")


Análise da Variável Target:
Distribuição das classes:
classificacao
1    3600
0    3600
Name: count, dtype: int64

Proporção das classes:
classificacao
1    0.5
0    0.5
Name: count, dtype: float64
🔎 Dataset Fake.Br detectado — invertendo para o padrão (1=real, 0=fake).

Distribuição com labels de texto:
label_texto
fake    3600
real    3600
Name: count, dtype: int64

Classe 1: 3600 amostras (50.0%) - Balanceado

Classe 0: 3600 amostras (50.0%) - Balanceado


Essa análise da variável classificacao mostrou que o dataset possui uma distribuição equilibrada entre as classes “fake” e “real”, o que é ideal para o treinamento do modelo. A criação da coluna label_texto facilita a leitura e visualização dos resultados nos gráficos seguintes. Caso houvesse desbalanceamento, seria necessário aplicar técnicas de balanceamento como oversampling ou undersampling.

In [26]:
print("\nChecando correspondência:")
display(df[['classificacao', 'REAL', 'label_texto']].head(5))



Checando correspondência:


,classificacao,REAL,label_texto
0,1,0,fake
1,1,0,fake
2,1,0,fake
3,1,0,fake
4,1,0,fake


In [40]:
# Analisar features linguísticas
print("Análise de Features Linguísticas")
print("="*50)

# Features numéricas (excluindo texto e IDs)
features_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
features_numericas = [f for f in features_numericas if f not in ['classificacao', 'REAL']]

print(f"Features disponíveis: {len(features_numericas)}")
print(f"Lista: {features_numericas[:10]}...")  # Primeiras 10

# Correlação com o target
correlacoes = {}
for feature in features_numericas:
    corr = df[feature].corr(df['REAL'])
    correlacoes[feature] = corr

# Top 10 features mais correlacionadas
top_features = sorted(correlacoes.items(), key=lambda x: abs(x[1]), reverse=True)[:10]
print(f"\nTop 10 features mais correlacionadas com fake/real:")
for feature, corr in top_features:
    print(f"  {feature:30}: {corr:+.3f}")

Análise de Features Linguísticas
Features disponíveis: 21
Lista: ['n_tokens', 'n_palavras_sem_pontuacao', 'n_tipos', 'n_links_na_noticia', 'n_palavras_maiusculas', 'n_verbos', 'n_verbos_subjuntivo_imperativo', 'n_substantivos', 'n_adjetivos', 'n_adverbios']...

Top 10 features mais correlacionadas com fake/real:
  diversidade                   : -0.810
  n_tipos                       : +0.721
  n_substantivos                : +0.694
  n_caracteres                  : +0.683
  n_palavras_sem_pontuacao      : +0.682
  n_tokens                      : +0.680
  n_verbos                      : +0.644
  n_verbos_modais               : +0.613
  n_adjetivos                   : +0.602
  n_adverbios                   : +0.564


## 5.Preparação para Machine Learning

In [42]:
# Preparar dados para ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Selecionar features e target
X = df[features_numericas]
y = df['REAL']

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Dados preparados:")
print(f"Treino: {X_train.shape[0]} amostras")
print(f"Teste: {X_test.shape[0]} amostras")
print(f"Features: {X_train.shape[1]} variáveis")

Dados preparados:
Treino: 5760 amostras
Teste: 1440 amostras
Features: 21 variáveis


## 6. Salvamento dos Dados Processados

Salvar os dados pré-processados para uso nos notebooks seguintes.

In [43]:
# Salvar dados processados
print("Salvando dados processados...")

# Verificar arquivos existentes
existing_files = os.listdir("../data/processed")
print(f"Arquivos existentes em ../data/processed: {existing_files}")

# Estratégia: manter o original e criar versões processadas
print("\nEstratégia de salvamento:")
print("   - fakeBr_dataset.csv (original) → mantido")
print("   - fakebr_with_labels.csv (+ colunas REAL, label_texto) → novo")
print("   - X_train/test, y_train/test → para ML")

# Salvar dataset com as novas colunas (REAL, label_texto)
df.to_csv("../data/processed/fakebr_with_labels.csv", index=False, encoding="utf-8")
print("Dataset com labels criados salvo em: ../data/processed/fakebr_with_labels.csv")

# Salvar dados de treino e teste separadamente (só features numéricas)
X_train.to_csv("../data/processed/X_train_fakebr.csv", index=False)
X_test.to_csv("../data/processed/X_test_fakebr.csv", index=False)
y_train.to_csv("../data/processed/y_train_fakebr.csv", index=False)
y_test.to_csv("../data/processed/y_test_fakebr.csv", index=False)

print("Dados de treino/teste salvos:")
print("   - X_train_fakebr.csv (features numéricas)")
print("   - X_test_fakebr.csv") 
print("   - y_train_fakebr.csv (target REAL)")
print("   - y_test_fakebr.csv")

# Mostrar diferenças importantes
print(f"\nResumo das modificações:")
print(f"   - Coluna 'classificacao' original: {df['classificacao'].value_counts().to_dict()}")
print(f"   - Nova coluna 'REAL' (invertida): {df['REAL'].value_counts().to_dict()}")
print(f"   - Nova coluna 'label_texto': {df['label_texto'].value_counts().to_dict()}")

print(f"\nResumo final:")
print(f"Dataset FakeBR pré-processado com sucesso!")
print(f"{df.shape[0]} amostras, {len(features_numericas)} features numéricas")
print(f"Classes balanceadas: 50/50 fake/real")
print(f"Arquivos salvos:")
print(f"  fakebr_with_labels.csv → para análise de fairness")
print(f"   X/y_train/test → para treinamento de modelos")
print(f"\nPróximo passo: Execute o notebook 03_fairness_analysis.ipynb")

Salvando dados processados...
Arquivos existentes em ../data/processed: ['.gitkeep', 'fakeBr_dataset.csv']

Estratégia de salvamento:
   - fakeBr_dataset.csv (original) → mantido
   - fakebr_with_labels.csv (+ colunas REAL, label_texto) → novo
   - X_train/test, y_train/test → para ML
Dataset com labels criados salvo em: ../data/processed/fakebr_with_labels.csv
Dados de treino/teste salvos:
   - X_train_fakebr.csv (features numéricas)
   - X_test_fakebr.csv
   - y_train_fakebr.csv (target REAL)
   - y_test_fakebr.csv

Resumo das modificações:
   - Coluna 'classificacao' original: {1: 3600, 0: 3600}
   - Nova coluna 'REAL' (invertida): {0: 3600, 1: 3600}
   - Nova coluna 'label_texto': {'fake': 3600, 'real': 3600}

Resumo final:
Dataset FakeBR pré-processado com sucesso!
7200 amostras, 21 features numéricas
Classes balanceadas: 50/50 fake/real
Arquivos salvos:
  fakebr_with_labels.csv → para análise de fairness
   X/y_train/test → para treinamento de modelos

Próximo passo: Execute o no